Importing Libraries:

-Pandas

-Sklearn

-Scikit

-Numpy

In [100]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

Creating Dataframe as `dataFrame`

In [103]:
dataFrame = pd.read_csv('FRAFirm.csv')
dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38759 entries, 0 to 38758
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FIRM      38759 non-null  int64  
 1   class     38759 non-null  int64  
 2   Start     38759 non-null  int64  
 3   End       38759 non-null  int64  
 4   Length    38759 non-null  int64  
 5   Night     38759 non-null  int64  
 6   Gap       38759 non-null  int64  
 7   WS        38759 non-null  int64  
 8   idx       38759 non-null  int64  
 9   Base      38759 non-null  float64
 10  StartAdj  38759 non-null  float64
 11  LenAdj    38759 non-null  float64
 12  Comp      38759 non-null  float64
 13  Trans     38759 non-null  float64
 14  Press     38759 non-null  float64
 15  p1s       38759 non-null  int64  
 16  p1l       38759 non-null  int64  
 17  p2s       38759 non-null  int64  
 18  p2l       38759 non-null  int64  
 19  MalAdj    38759 non-null  int64  
 20  NFZ       38759 non-null  in

Creating a K Fold Cross Validation as `kfold`

In [106]:
kfold = KFold(n_splits = 10, shuffle = True, random_state=42)
model = LogisticRegression(max_iter=200)
accuracy_scores = []

Seperating column `class` from the dataset to create the `y` series 

In [109]:
y = dataFrame['class']
X = dataFrame[[col for col in dataFrame.columns if col != 'class']]
X.info()
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38759 entries, 0 to 38758
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   FIRM      38759 non-null  int64  
 1   Start     38759 non-null  int64  
 2   End       38759 non-null  int64  
 3   Length    38759 non-null  int64  
 4   Night     38759 non-null  int64  
 5   Gap       38759 non-null  int64  
 6   WS        38759 non-null  int64  
 7   idx       38759 non-null  int64  
 8   Base      38759 non-null  float64
 9   StartAdj  38759 non-null  float64
 10  LenAdj    38759 non-null  float64
 11  Comp      38759 non-null  float64
 12  Trans     38759 non-null  float64
 13  Press     38759 non-null  float64
 14  p1s       38759 non-null  int64  
 15  p1l       38759 non-null  int64  
 16  p2s       38759 non-null  int64  
 17  p2l       38759 non-null  int64  
 18  MalAdj    38759 non-null  int64  
 19  NFZ       38759 non-null  int64  
 20  AFZ       38759 non-null  in

Create the K number of Folds and start cross validation to find coefficients for dataset

In [112]:
for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

A:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
A:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

In [114]:
average_accuracy = np.mean(accuracy_scores)
print(f'Average Accuracy: {average_accuracy:.2f}')

Average Accuracy: 0.93
